## A concept for forecasting inflation with online listed item prices
Can we use online prices as a predictor of a nation's inflation and CPI?

Resources online indicates that not only is this possible, but it has been done the past decade by those who has the right tools - http://www.mit.edu/~afc/papers/Cavallo_Online_Offline.pdf

While it isn't entirely surprising that online and offline prices are similar.
finding the correct data points in an ever growing pool of resources is the challenge that this model attempts to answer.

Technolgy used: Jupyter Notebook, plot.ly, Python, d3js, Javascript
<br><br><i>
    <b>Output</b> = df
</i>

In [ ]:
#Load Libraries for offline use
# Dummy edit
import os                                           # Fundamental file management libraries

import numpy as np                                  # Base Array library used by Pandas
import pandas as pd                                 # Pandas Matrix library

import scipy as sp                                  # Required as the baseline data science module


try:
    from StringIO import StringIO                   # Formulating a string as a filestream
except ImportError:
    from io import StringIO
    
import plotly.tools as tls                          # Auxiliary Tools

from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
                                                    # plotly for offline use (i.e. no service fee)
    
from plotly.graph_objs import *                     # Different chart types
init_notebook_mode()                                # Notebook 

import cufflinks as cf                              # Bridge from DataFrames to Plotly
cf.go_offline()                                     # Required to use plotly offline (no account required).

from sklearn import preprocessing                   # For natrix normalization

# import seaborn as sns                             # For gradient color scales

from IPython.display import display, HTML           # Formatting for Dataframes

In [ ]:
START_OFFSET=-150
END_OFFSET=0
### Formatting 
pd.options.display.float_format = '{:,.2f}'.format

categorybasepath=os.getcwd()
if 'historical_price' not in categorybasepath:
    categorybasepath = categorybasepath +'/'+ 'willzjc.github.io/python/historical_price/ref/categories/'
else:
    categorybasepath = categorybasepath.split('historical_price')[0] + 'historical_price/ref/categories/'


categorybasepath=categorybasepath.replace('\\','/')
categories=list(set([x[0].replace(categorybasepath+'\\','').split('categories')[1].replace('/','').split('\\')[0].strip() 
                     for x in os.walk(categorybasepath)]))

categories = [c for c in categories if (
              'ipynb_checkpoints' not in c and
              'Misc' not in c and not 'archive' in c
    )]

# print '\n'.join([x for x in categories if len(x)>0])

###  Priming data into model
Nothing too complex, loading data into matrices
<br><br><i>
    <b>Output</b> = df, readFiles()
</i>

In [ ]:
#Reads files

global category,basepath

def readfiles(categorybasepath):
    global category,basepath
    bol_recursive=False
    df=None
    global category

    category='Food and non-alcoholic beverages'
    category='Furnitures'
#     category='Alcohol'
#     category='Clothing and footwear'
    basepath=categorybasepath+'/'+category+'/'
    
    files=[]

    for str_dirname, lst_subdirs, lst_files in os.walk(basepath):
        if not bol_recursive:
            while len(lst_subdirs) >= 0:

                for file in lst_files:
                    if '.csv' in file and not 'corr' in file and not 'fileread' in file:
#                         print 'Reading:',file
                        with open(basepath+file,'rb') as f:
                            buffer=f.read()
                            files.append(buffer)
                            f.close()
                
                if len(lst_subdirs) > 0: 
                    lst_subdirs.pop()
                    if len(lst_subdirs)==0:
                        break
                else:
                    break
                    
    for f in files:

        headers = {}
        buffer=[]
        headermode = True
        for line in f.split('\n'):
            if headermode and 'Date,' in line:
                headermode=False
            elements=line.strip().split(',')
            if len(elements) < 3 and len(elements) > 1:         # filter out header info
                headers[elements[0].strip()]=elements[1].strip()
            else:
                linein=False
                if len(elements)>1 and not headermode and '0.00 USD,0.00 USD' not in line:
                    buffer.append(line)
                    linein=True
                # print f,linein,line

        # Read file stream CSV
        currentdf = pd.read_csv(StringIO('\n'.join(buffer)))

        # Replace Strings
        currentdf = currentdf.replace('\sUSD', '', regex=True).apply(pd.to_numeric, errors='ignore')
        try:
            if df==None:
                df=pd.DataFrame(columns=['date'])
                df['date'] = currentdf['Date']
        except Exception as e:
            # TODO
            do_nothing=True

        df[headers['Keywords']] = currentdf['Average Selling Price']
        df[headers['Keywords']+"_sales"] = currentdf['Total Sales']
        df[headers['Keywords']+"_weighting"] = currentdf['Total Sales'] /  currentdf['Average Selling Price']

    return df

df = readfiles(categorybasepath)
raw_df=df.copy()
df.head(3)

### First pass of data enrichment
1. Get all columns which are base values
2. Find all columns which are other metrics
3. Transforming raw matrix into a summary matrix
<br><br><i>
    <b>Output</b> = df, pricecolumns, salescolumns, weighcolumns
</i>

In [ ]:
series_summary = df.copy()

series_summary=series_summary.sum()[[c for c in series_summary.columns if 'date' not in c]].astype(int)
dfs=pd.DataFrame(series_summary,columns=['value'])

# Get all row names which are metrics
metrics=[]
ml=list(filter(lambda x: keyword in x, dfs.index.values) for keyword in ['sales','weigh'])
for l in ml: metrics=metrics+l

# Get all rows which are base values
items=list(set(dfs.index.values) -  set(metrics))

# Worked out mean and sum

def getColumns(df,exclude_cols=None):
    if not exclude_cols==None:
        exclude_cols.append('date')
    else:
        exclude_cols=['date']
    return [x for x in df.columns if x not in exclude_cols]

# Getting column names of the different dimensions
pricecolumns = [c for c in df.columns if (not 'date' in c and not 'sales' in c and     'weight' not in c)]
salescolumns = [c for c in df.columns if (not 'date' in c and     'sales' in c and not 'weight' in c)]
weighcolumns = [c for c in df.columns if (not 'date' in c and not 'sales' in c and     'weight' in c)]


transformed_df=pd.DataFrame(columns=['item','avg_price','units_sold','revenue'])

for i,item in enumerate(set(items)):                                    # Transform information to a new summary frame
    avg_price  = item 
    units_sold = item +'_'+'weighting' 
    revenue    = item +'_'+'sales'
    transformed_df.loc[len(transformed_df)] = [item
           ,dfs.loc[dfs.index.isin([avg_price])]['value'][0]
           ,dfs.loc[dfs.index.isin([units_sold])]['value'][0]
           ,dfs.loc[dfs.index.isin([revenue])]['value'][0]
          ]

# Auxiliary Styling Function
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

# transformed_df=transformed_df.set_index('item')
transformed_df=transformed_df.sort_values(by=['revenue'],ascending=False)
transformed_df['avg_price']=transformed_df['revenue'] / transformed_df['units_sold']

# Aggregating for Total
total_df = pd.DataFrame(transformed_df[filter(lambda x: 'item' not in x, transformed_df.columns)].sum(),columns=['total']).astype(int)

total_df.loc[len(total_df)]=len(items)
total_df.index.values[len(total_df)-1] = 'Category Count'

#Formatting
for c in ['units_sold']:
    transformed_df[c]=transformed_df[c].map('{:,}'.format)

for c in ['revenue','avg_price']:
    transformed_df[c]=transformed_df[c].map('${:,.0f}'.format)

# Final formatting
total_df['total']=total_df['total'].map('{:,}'.format)

#========================
print('Aggregate Stats of categories')
#Display Categorical Summary
display(HTML(transformed_df.to_html(index=False)))
print('\n\nTotal of all')
#Display Total Aggregate Summary
display(total_df)

# First Plot - Seeing the data first hand
No changing or modification of data yet

In [ ]:
# Resampling and interpolate

def interpolate_df(df, columns=None,frequency='M'):

    sample_columns=df.columns

    if not columns == None:
        sample_columns = list(columns)
        if 'date' not in sample_columns:
            sample_columns.append('date')

    ndf = df.copy()[sample_columns]

    indexer = 'date'  # Only do this when the column date exists, set index column as date
    if indexer in df.columns:
        ndf['date'] = pd.to_datetime(df['date'])
        ndf = ndf.set_index(pd.DatetimeIndex(df['date']))

    ndf = ndf.resample(frequency).mean()
    ndf = ndf.resample('D')
    tsint = ndf.interpolate(method='cubic')
    return tsint

frequency='7D'

# Show Price (interlolated to 1M)
title='<b>Average Price ($)</b><br>Outliers not filtered<br>Interpolated and bucketing interval set to: <i>%s</i>'%(frequency)
interpolate_df(df,columns=pricecolumns,frequency=frequency).iplot(title=title)

# Show Units Sold (interlolated to 1M)
title="<b>Units Sold</b><br>Outliers not filtered"
interpolate_df(df,columns=weighcolumns,frequency=frequency).iplot(title=title)

# Show Sales Revenue (interlolated to 1M)
title="<b>Sales revenue of items ($)</b><br>Outliers not filtered"
interpolate_df(df,columns=salescolumns,frequency=frequency).iplot(title=title)

# Data Normalization
This section 
1. Normalizes data
2. Removes any outliters above a certain percentile
3. Smoothes out chart via more suitable bucketing interval

In [ ]:
#### Percentiel threshold
percentile=0.90

for c in df.columns:
    if c not in ['date']:
        q = df[c].quantile(percentile)
        df[c] = df[df[c] < q][c]

# df.iplot(y=getColumns(df),title='<b>Outliers Filtered</b><br>Period bucketing frequency unaltered')

# interpolate, spine-smooth, and then plot 

interpolate_df(df,pricecolumns).iplot(title='<b>Price - Outliters Filtered ('+str(int(percentile*100))+ '%)</b><br>Interpolated and bucketing interval set to 1 month')
interpolate_df(df,salescolumns).iplot(title='<b>Revenue - Outliters Filtered</b><br>Interpolated and bucketing interval set to 1 month')
interpolate_df(df,weighcolumns).iplot(title='<b>Items Sold - Outliters Filtered</b><br>Interpolated and bucketing interval set to 1 month')

### In this next step - we normalize all prices to be its own scale
Bias is given to those categories which had a higher volume
1. Average price of each category product accounted for
2. As well as being weighted how many items of each category of the product is sold

In [ ]:
##### Make a copy of variables first
input_df = df.copy()
input_df = input_df.interpolate(method='linear', axis=0).ffill().bfill()

##### Normalization for price #######

# norm_columns = pricecolumns + weighcolumns                       # Normalize both price and weighting
norm_columns = []                                                  # Normalize both price and weighting

for c in (pricecolumns + weighcolumns):
    if not 'date' in c:
        norm_columns.append(c)

display(input_df.head())
x = input_df[norm_columns].values                                  # returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()                      # Scaling
x_scaled = min_max_scaler.fit_transform(x)                         # Fits curve
ndf = pd.DataFrame(x_scaled)


#### Normalized Values
ndf['date']=pd.to_datetime(input_df['date'])
ndf=ndf.set_index(pd.DatetimeIndex(input_df['date']))

                                                                   # Fixes up columns
ndf=ndf.drop(columns=['date'])
ndf.columns=(norm_columns)

                                                                   # Assign weighting based on sales
for product in pricecolumns :
    if product not in 'mean':
        ndf[product]=ndf[product] * ndf[product + "_weighting"]    # Weighting Calculation
#         ndf[product]=ndf[product] # * ndf[product + "_weighting"]    # No Weighting Calculation
        ndf=ndf.drop(columns=[product + "_weighting"])

ndf['mean']=ndf[pricecolumns].mean(axis=1)
        
ndf=interpolate_df(ndf)                                           # Interpolate first
fig = tls.make_subplots(rows=2, cols=1, shared_xaxes=True)         # Sub Plotting, specify how many charts

# for col in [c for c in getColumns(ndf) if c not in ['mean']]:
for col in pricecolumns :
    fig.append_trace({'x': ndf.index, 'y': ndf[col], 'type': 'scatter', 'name': col}, 1, 1)
for col in ['mean']:
    fig.append_trace({'x': ndf.index, 'y': ndf[col], 'type': 'bar', 'name': col}, 2, 1)


fig.layout.title='Weighted Normalized prices chart in comparison to mean price'
iplot(fig) 


## CPI stats
Sourced from rba.gov.au, a csv file can be downloaded and used for our model's benchmark and reference

In [ ]:
#Next is to get CPI
cpi=pd.read_csv(basepath+'AUCPI',delimiter='\t')
title='Obtain CPI for the next step, including each point\'s derivative'

cpi['diff'] = cpi.CPI.diff() # Calculating difference from previous year
cpi['diff'] = 100* cpi['diff'] / ((cpi['CPI'] + cpi['CPI'].shift(-1))/2)
cpi.index=cpi['date']

# Plotting two types of charts
def plotdouble(df,metric1,metric2,color1='orange',color2='green'):
    fig1 = df.iplot(columns=[metric1], kind='bar',asFigure=True,width=0.1,color=color1)   
    fig2 = df.iplot(columns=[metric2],  kind='line',secondary_y=[metric2], asFigure=True,colors=color2,width=5)
    fig2['data'].extend(fig1['data'])
    fig2.layout.title=title
    
    return fig2

iplot(plotdouble(cpi,'diff','CPI'))

In [ ]:
# Interpolate
icpi=cpi.set_index(pd.DatetimeIndex(cpi.index))
icpi=icpi.resample('D').mean()
# icpi.CPI=icpi.CPI.resample('D').mean()
# icpi['CPI'].iplot()
# print icpi.interpolate()
# combined_df=pd.concat([cpi,df],axis=1)
# combined_df
# interpolation

icpi.CPI=icpi.CPI.interpolate(type='spline')
# icpi['CPI'].iplot(title='First interpolated spline for CPI')

icpi['diff'] = 0
icpi['diff'] = icpi.CPI.diff() # Calculating difference from previous year
icpi['diff'] = 100* icpi['diff'] / ((icpi['CPI'] + icpi['CPI'].shift(-1))/2)

# Filter out null
icpi=icpi[icpi['diff'].notnull()]

# sets index as the date for prices
# also time serializes dataframe so as to allow concatenation

#Either use prices or normalized prices
prices=df.set_index(pd.DatetimeIndex(df['date'])).drop(columns=['date'])
normalized_prices=ndf

#Combine Prices
cdf=pd.concat([icpi,normalized_prices],axis=1,join='inner')
corr=cdf['diff'].corr(cdf['mean'])

# Get Doubleplot figure
fig=plotdouble(cdf,metric1='mean',metric2='diff')
fig.layout.title='Final Result'

mean_renamed='Price predictor'
cpi_renamed ='CPI %'
this_cdf=cdf.copy().rename(columns = {'mean': mean_renamed,'diff':cpi_renamed})
fig_resampled=plotdouble(this_cdf.resample('M').mean(),metric1=cpi_renamed,metric2=mean_renamed)
fig_resampled.layout.title='<b>'+ category + '</b><br>'+'Mean Price and CPI - Monthly' + '<br>Correlation: '+str(round(corr,2))

iplot(fig_resampled)




### Time shift
While the above looks correct, online prices are a lot more adaptive than RBA rates. 
Hence a timeshift is done to, each step is done per day, and the highest correlating shift is automatically chosen

In [ ]:
def timeshift_series_plot(input_icpi,normalized_prices,offset,orientation=0,printoffset=True,sampling_rate='M',color1='orange',color2='blue'):
    if offset==0 :printoffset=False
        
    global mean_renamed
    global cpi_renamed
    
    icpi=input_icpi.copy()
    
    ref_icpi=icpi.copy()
    ref_normalized_prices=normalized_prices.copy()
    title='<b>'+ category + '</b><br>'+'Result - Monthly. '
    if orientation == 0:                                                        # Time shift and filter out NaN values
        title=title+' CPI shift: '
        icpi=icpi.copy().shift(offset)
        icpi=icpi[icpi.CPI.notnull()]
        
#         if printoffset: print ref_icpi.head(),'\nShifted:\n', icpi.head()

    else:
        title=title+' Price shift: '
        normalized_prices=normalized_prices.shift(offset)
        normalized_prices=normalized_prices[normalized_prices['mean'].notnull()]
#         if printoffset: print ref_normalized_prices.head(),'\n\nShifted:\n', normalized_prices.head()
                                                                                
    cdf=pd.concat([icpi,normalized_prices],axis=1)                              # Combine Dataframes: CPI and Prices
    
    if not sampling_rate == 'M':                                                # If we want to change sampling rate
        cdf=cdf.resample(sampling_rate).mean().interpolate(kind='spine')    

    corr=cdf['diff'].corr(cdf['mean'])                                          # Calculate Correlation
    title= title + str(offset) +' days. Correlation: '+str(round(corr,2))

    
#     fig=plotdouble(cdf,metric1='mean',metric2='diff')                           

    cdf=cdf.rename(columns = {'mean': mean_renamed,'diff':cpi_renamed})
    fig_resampled=plotdouble(cdf.resample(sampling_rate).mean(),metric1=cpi_renamed,metric2=mean_renamed,color1=color1,color2=color2)
    fig_resampled.layout.title=title
    
    iplot(fig_resampled)
    return fig_resampled


#find correlation of range
correlation_matrx=pd.DataFrame(columns=['offset','correlation'])
for i in range(START_OFFSET,END_OFFSET):                                                      # Looping through spectrum
    licpi=icpi.copy().shift(i)
    licpi=licpi[licpi.CPI.notnull()]

    cdf=pd.concat([licpi,normalized_prices],axis=1,join='inner')
    corr=cdf['diff'].corr(cdf['mean'])   # Calculate Correlation
    
    correlation_matrx.loc[len(correlation_matrx)]=[i,corr]

correlation_matrx.iplot(y='correlation',x='offset',title='<b>'+ category + '</b><br>'+'Days Offset Correlation Spectrum')


In [ ]:
scdf=correlation_matrx.copy()
highest_offset_row = scdf.loc[scdf['offset']> -388].loc[scdf['offset']<55].sort_values(['correlation'],ascending=False).head(n=1)
h_offset = highest_offset_row['offset'].values[0].astype(int)

double_chart=timeshift_series_plot(input_icpi=icpi,normalized_prices=normalized_prices,offset=h_offset,orientation=0
                      ,sampling_rate='M'
                      ,color1='yellow'
                      ,color2='purple'
                     )
# print scdf
# for r in scdf.iterrows():
#     print r[1]['offset'],r[1]['correlation']


In [ ]:
def plot_final(icpi,normalized_prices,recalc_correlation=False,h_offset=h_offset):
    bar_label='CPI % Change'
    original_curve='Predictor'
    new_curve='Time Shifted Predictor'

    data1=icpi.copy()['diff']                                                              # CPI 
    data1=data1.to_frame()
    data1=data1.rename(columns = {'diff': bar_label})

    data2=normalized_prices.copy().shift(-h_offset)['mean'].to_frame()                     # Shifted Price Curve
    data2=data2.rename(columns = {'mean': new_curve})
    data2=data2[data2[new_curve].notnull()]
    
    h_offset_corr = highest_offset_row['correlation'].values                               # Correlation
    if recalc_correlation:
        h_offset_corr=data2[new_curve].corr(icpi['diff'])


    data3=normalized_prices.copy()['mean'].to_frame()                                      # Prices
    data3=data3.rename(columns = {'mean': original_curve})

    combined_frame=pd.concat([data1,data2,data3[original_curve]],axis=1)      # Combined
    

        
    title='<b> %s </b><br>Interval: Monthly    Offset: %s    Correlation: %s'%(category,h_offset,round(h_offset_corr,2))
    color1='orange'
    color2='green'

    # Comparing a substring of 1 list to another list
    # Lambda combined with any()
    # combined_frame.iplot(columns=filter(lambda x: not any(n in x for n in ['diff','CPI']), combined_frame.columns))

#     combined_frame=combined_frame.resample('W').mean().interpolate(kind='spine')   
    combined_frame=combined_frame.resample('W').mean()

    fig1 = combined_frame.iplot(columns=[bar_label], kind='bar',asFigure=True,width=5,color=color1)   

    fig2 = combined_frame.iplot(columns=[original_curve]
        ,kind='line',secondary_y=[original_curve]
        ,asFigure=True,colors=['green'],width=5,dash='dot'
        )

    fig3 = combined_frame.iplot(columns=[new_curve]
        ,kind='line',secondary_y=[new_curve]
        ,asFigure=True,colors=['purple'],width=5
        )

    fig2['data'].extend(fig1['data'])
    fig3['data'].extend(fig2['data'])
    fig3.layout.title=title
    iplot(fig3)

plot_final(icpi,normalized_prices)
    
def find_highest_correlation(icpi,normalized_prices,offset_start=-250,offset_end=0,target_column='mean'):
#     print normalized_prices.to_frame().columns
    target_frame=normalized_prices.copy().to_frame()
    
    correlation_matrx=pd.DataFrame(columns=['offset','correlation'])
    corrmax=0
    corroffset=0
    for i in range(offset_start,offset_end):
        licpi=icpi.copy().shift(i)
        licpi=licpi[licpi['diff'].notnull()]
        cdf=pd.concat([licpi,normalized_prices],axis=1,join='inner')        
        corr=cdf['diff'].corr(cdf[target_column])                                  # Calculate Correlation
        correlation_matrx.loc[len(correlation_matrx)]=[i,corr]
        if corrmax<corr:
            corroffset=i
            corrmax=corr
    
    correlation_matrx=correlation_matrx.sort_values('correlation',ascending=False)
    h_c_offset = correlation_matrx.head(1)['offset'].values[0]
    target_frame=target_frame.shift(-int(corroffset))
    target_frame=target_frame[target_frame[c].notnull()]
#     target_frame=target_frame[target_frame[c].notnull()]

    return correlation_matrx,target_frame

In [ ]:
print 'Highest Offset is: %s'%(h_offset)
ref_normalized_prices=normalized_prices.copy()
ref_cpi = icpi.copy()['diff'].to_frame()
ref_cpi = ref_cpi.shift(h_offset)
#
individual_correlation_df=pd.DataFrame(columns=['item','correlation','type','highest-c','offset'])

acceptable_columns=[]
shifted_frames=[]
for c in [x for x in ref_normalized_prices.columns if  'mean' not in x]:
    correlation=ref_normalized_prices[c].corr(ref_cpi['diff'])
    status=''

    cm,tf=find_highest_correlation(ref_cpi,ref_normalized_prices[c],offset_start=START_OFFSET,offset_end=END_OFFSET,target_column=c)
    highest_c  = cm.head(1)['correlation'].values[0]
    h_c_offset = cm.head(1)['offset'].values[0]
    threshold=0.55
    if correlation > threshold:
        acceptable_columns.append(c)
        status='immediate'
    elif highest_c > threshold:
        
#         if int(h_c_offset)
        
        acceptable_columns.append(c)
        status='long range'
        
    
    shifted_frames.append(tf)
    individual_correlation_df.loc[len(individual_correlation_df)]=[c,correlation,status,highest_c,h_c_offset]

cat_summary=transformed_df.copy()
cat_summary=cat_summary.set_index('item')
individual_correlation_df=individual_correlation_df.set_index('item')
individual_correlation_df=pd.concat([cat_summary,individual_correlation_df],axis=1)
# individual_correlation_df=individual_correlation_df.sort_values('correlation',ascending=False)

fcdf=None
fcdf_init=False
for f in shifted_frames:
#     display(f)
#     display(icpi)
    fcpi=pd.concat([f,icpi['diff']],axis=1,join='inner')
    product = f.columns[0]
    print product
    fcorr=fcpi[product].corr(fcpi['diff']) 
    fcpi.iplot(title='Correlation=%s'%(fcorr))
    if not fcdf_init:
        fcdf=f
        fcdf_init=True
    else:
        fcdf = pd.concat([f,fcdf],axis=1)
        

ref_normalized_prices['mean']=ref_normalized_prices[acceptable_columns].mean(axis=1)
# plot_final(icpi=icpi,normalized_prices=ref_normalized_prices[acceptable_columns+['mean']],recalc_correlation=True)

fcdf['mean']=((fcdf[acceptable_columns].mean(axis=1)).to_frame())

display(individual_correlation_df.sort_values('highest-c',ascending=False))
fcdf=fcdf.rename(columns = {1 : 'mean'})
plot_final(icpi=icpi,normalized_prices=fcdf[acceptable_columns+['mean']],recalc_correlation=True)